In [1]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [4]:
# Grayson's path
train_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\nineteen-class\\data\\train\\"
test_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\nineteen-class\\data\\test\\"

# Will's path (comment out if not in use)
# train_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\train\\"
# test_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\test\\"

# Function to load and concatenate CSV files from a list of file paths
def load_and_concat(files, sample_fraction=None):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        if sample_fraction:
            df = df.sample(frac=sample_fraction, random_state=42)  # Random sampling
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# List all CSV files in the train and test folders
train_files = glob.glob(train_folder + "*.csv")
test_files = glob.glob(test_folder + "*.csv")

# Load and concatenate training and testing data
train_df = load_and_concat(train_files, sample_fraction=0.1)  # Use 10% of the data
test_df = load_and_concat(test_files, sample_fraction=0.1)

In [5]:
# Combine target labels into a single column for multi-class classification
# Assuming the labels are one-hot encoded
target_columns = ['spoofing', 'benign', 'MQTT_DDoS_Connect_Flood', 'MQTT_DoS_Connect_Flood', 
    'MQTT_DDoS_Publish_Flood', 'MQTT_DoS_Publish_Flood', 'MQTT_Malformed_Data',
    'Recon_OS_Scan', 'Recon_Ping_Sweep', 'Recon_Port_Scan', 'Recon_VulScan',
    'DoS_ICMP', 'DoS_SYN', 'DoS_TCP', 'DoS_UDP',
    'DDoS_ICMP', 'DDoS_SYN', 'DDoS_TCP', 'DDoS_UDP'
]
feature_columns = [col for col in train_df.columns if col not in target_columns]

# Separate features and labels
X_train = train_df[feature_columns]
y_train = train_df[target_columns]
X_test = test_df[feature_columns]
y_test = test_df[target_columns]

# Convert one-hot encoded labels to integer labels for y_train and y_test
y_train_labels = y_train.idxmax(axis=1)
y_test_labels = y_test.idxmax(axis=1)

In [6]:
# Encode the labels to numerical values
label_encoder = LabelEncoder()
y_train_labels = label_encoder.fit_transform(y_train_labels)
y_test_labels = label_encoder.transform(y_test_labels)

# Create a scaler object
scaler = MinMaxScaler()

# Fit the scaler to the training features and transform both training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Build the DNN model
model = Sequential()
model.add(Dense(128, input_dim=len(feature_columns), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(target_columns), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Train the model
model.fit(X_train_scaled, y_train_labels, epochs=25, batch_size=128, validation_split=0.2)

# Make predictions
y_pred_probs = model.predict(X_test_scaled)
y_pred_labels = y_pred_probs.argmax(axis=1)

# Evaluate the model
print(confusion_matrix(y_test_labels, y_pred_labels))
print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

Epoch 1/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8589 - loss: 0.4608 - val_accuracy: 0.0000e+00 - val_loss: 30.9716
Epoch 2/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9211 - loss: 0.2140 - val_accuracy: 0.0000e+00 - val_loss: 43.6601
Epoch 3/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9237 - loss: 0.2034 - val_accuracy: 0.0000e+00 - val_loss: 58.2393
Epoch 4/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9238 - loss: 0.1998 - val_accuracy: 0.0000e+00 - val_loss: 75.0417
Epoch 5/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9242 - loss: 0.1982 - val_accuracy: 0.0000e+00 - val_loss: 85.7235
Epoch 6/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9241 - loss: 0.1971 - val_accuracy: 1.3965e-05 - val_loss: 89.9159
Epoch 7/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9243 - loss: 0.1960 - val_accuracy: 0.0000e+00 - val_loss: 91.9411
Epoch 8/25
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step

c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\grays\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif